### Homework 3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Dataset

In [2]:
# We will use the lead scoring dataset Bank Marketing dataset.

data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"

In [3]:
df = pd.read_csv("data-hw3.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


#### Data preparation
* Check if missing values are present in the features.
* If there are missing values:
  * For categorical features, replace them with 'NA'
  * For numerical features, replace with 0.0.

In [4]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
numerical = [ 'number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score' ]

In [6]:
categorical = [ 'lead_source', 'industry', 'employment_status', 'location' ] 

In [7]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [8]:
df['lead_source'].value_counts()

lead_source
organic_search    282
social_media      278
paid_ads          264
referral          260
events            250
Name: count, dtype: int64

In [9]:
282+278+264+260+250

1334

In [10]:
len(df)

1462

In [11]:
1462-1334

128

In [12]:
# Fill missing values in categorical variables with 'NA'

df['lead_source'] = df.lead_source.fillna('NA')
df['industry'] = df.industry.fillna('NA')
df['employment_status'] = df.employment_status.fillna('NA')
df['location'] = df.location.fillna('NA')

In [13]:
# Check there are no missing values in categorical variables

df[ categorical ].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [14]:
# Fill missing values in numerical variables with 0.0

df['annual_income'] = df.annual_income.fillna(0.0)

In [15]:
# Check there are no missing values in numerical variables

df[ numerical ].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

In [16]:
# Check there are no missing values in the dataset
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

#### Question 1
What is the most frequent observation (mode) for the column 'industry'?

In [17]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [18]:
# The most frequent observation in 'industry' column is 'retail'.

#### Question 2

Create the correlation matrix for the numerical features.
What are the two features that have the biggest correlation?

In [19]:
df.corr(numeric_only=True)

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [20]:
# What are the 2 features that have the biggest correlation?

# interaction_count and lead_score 0.009888
# number_of_courses_viewed and lead_score -0.004879
# number_of_courses_viewed and interaction_count -0.023565
# annual_income and interaction_count 0.027036 (This pair has the largest correlation.)

#### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value 'converted' is not in your dataframe.

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [23]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [25]:
876+293+293, len(df)

(1462, 1462)

In [26]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [27]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [29]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score'],
      dtype='object')

#### Question 3

* Calculate the mutual information score between 'converted' and other categorical variables in the dataset.  Use the training set only.
* Round the scores to 2 decimals using round(score, 2). 

In [30]:
from sklearn.metrics import mutual_info_score

In [33]:
# A function to calculate the mutual info score between 'converted' in the training set

def mutual_info_converted_score(series):
    return round( mutual_info_score(series, y_train), 2)

In [34]:
# Calculate mutual information score between 'converted' in the training set with other categorical variables in the training set.

mi = df_train[ categorical ].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.04
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

In [35]:
# Which variable has the biggest mutual information score?  lead_source

#### Question 5

* Train a logistic regression.
* Include categorical variables using one-hot encoding.
* Fit the model on the training dataset.
  * To ensure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  * model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [36]:
from sklearn.feature_extraction import DictVectorizer

In [37]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[ categorical + numerical ].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[ categorical + numerical ].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [40]:
y_pred = model.predict_proba(X_val)[:, 1]

In [41]:
converted_decision = (y_pred >= 0.5)

In [42]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = converted_decision.astype(int)
df_pred['actual'] = y_val

In [43]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [46]:
df_pred.correct.mean()

np.float64(0.6996587030716723)

In [47]:
0.6996587030716723 - 0.64, 0.6996587030716723 - 0.74

(0.059658703071672314, -0.04034129692832766)